In [2]:
import numpy as np
import pandas as pd
import os

from common import OUTPUTPATH
from models import rule_fit_pcc
import data

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/var/folders/c_/6873d_hj27z4bhy1l9x1tggnj8s0wd/T/ipykernel_1849/2424450009.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


This notebook refit the models in the all dataset

Fit all data takes ~3 mim under 2.6 GHz 6-Core Intel Core i7

In [3]:
rule_fit_pcc.fit(data.x1, data.y)

In [4]:
fitted_model = rule_fit_pcc.fitted_
mophology = ['Sphere', 'Worm', 'Vesicle', 'Other']
for i in range(len(fitted_model)):
    print(mophology[i], 'Cs:', fitted_model[i].model.Cs)

Sphere Cs: [1]
Worm Cs: [1]
Vesicle Cs: [1]
Other Cs: [32]


Get the individual morphology rules

In [5]:
all_columns = data.predictors + data.targets
rule_sphere = rule_fit_pcc.fitted_[0].get_rules(all_columns)[['rule','type','coef','importance']]
rule_sphere.to_csv(os.path.join(OUTPUTPATH, 'Sphere_rules.csv'), index=False)
rule_sphere

,rule,type,coef,importance
5596,core_mw_total> 22332.4004 & corona_mw_total<= 12760.5298 & temp> 47.0 & mon_core_mw<= 156.595 & core_mw_total> 15534.3501 & conc> 16.1 & mon_corona_apol<= 42.472 & ratio_mass<= 0.2866 & clogp_core<= 1.37,rule,-3.888100,1.332257
11391,mon_core_apol<= 22.8716 & corona_mv_total<= 92.6556 & ph> 2.625 & corona_mw_total<= 15664.1104 & ratio_mass<= 0.325 & conc> 6.25 & core_mw_total> 27375.2002 & charged<= 0.5 & clogp_core<= 1.37,rule,-1.955195,0.621522
8878,core_mv_total> 135.5229 & core_mv_total<= 338.511 & dp_core> 105.0 & salt<= 0.05 & mon_core_psa> 37.18 & ratio_vol<= 0.3104 & clogp_core<= 1.1175 & dp_corona> 29.5 & corona_mw_total<= 10396.3247 & ratio_mass> 0.2342 & corona_mw_total<= 11925.2148,rule,-1.861998,0.555861
9017,dp_core> 96.5 & core_mv_total<= 510.1229 & mon_corona_apol<= 23.6109 & core_mw_total> 15703.46 & corona_mv_total<= 67.7817 & mon_corona_mv<= 1.344 & dp_corona<= 63.5 & clogp_core<= 1.37 & corona_mw_total<= 11925.2148,rule,-1.606697,0.508979
9594,ratio_vol<= 0.2233 & dp_core> 132.5 & mon_core_mv> 1.0413 & core_mw_total> 21611.625 & dp_corona<= 64.5 & mon_corona_apol<= 27.5791 & clogp_core<= 1.37 & mon_core_mv<= 1.1928,rule,-1.526991,0.493943
10892,core_mw_total> 30761.0801 & mon_corona_mv<= 2.3358 & conc> 15.5 & mon_corona_psa<= 215.065 & ratio_vol<= 0.2996 & mon_corona_mw> 163.9357 & clogp_core> 0.895 & clogp_corona<= -2.73 & conc> 16.1 & core_mw_total> 26510.7197,rule,-2.111469,0.446109
4231,corona_mv_total<= 79.0114 & mon_core_mv<= 1.3954 & corona_mw_total<= 12760.5298 & ratio_mass> 0.1338 & mon_corona_mw> 71.55 & temp> 47.0 & core_mw_total<= 43296.0391 & clogp_corona> -5.23 & ratio_vol<= 0.3115 & core_mv_total> 153.082 & conc> 14.5,rule,-1.413864,0.419645
14889,mon_core_mw<= 145.3096 & ratio_mass<= 0.4342 & core_mv_total<= 124.425 & mon_corona_mv<= 1.2652 & dp_corona<= 63.5 & corona_mw_total<= 6562.8699 & clogp_core<= 1.37,rule,-1.897467,0.392255
12630,dp_core> 95.0 & corona_mv_total<= 98.8696 & ratio_vol<= 0.2225 & ratio_mass<= 0.3023 & dp_corona<= 66.0 & clogp_core<= 1.1022 & dp_core> 87.0 & mon_corona_apol<= 27.5791 & clogp_core<= 1.37,rule,-0.962212,0.326188
3774,dp_corona<= 62.5 & clogp_core<= 1.18 & mon_core_psa<= 47.1448 & ratio_mass<= 0.3959 & core_mw_total> 16403.6704 & ratio_mass<= 0.2115 & mon_corona_apol<= 27.5791 & mon_core_mv> 1.186,rule,-1.225426,0.303337


In [6]:
all_columns = data.predictors + data.targets
rule_sphere = rule_fit_pcc.fitted_[0].get_rules(all_columns)[['rule','type','coef','importance']]
rule_sphere.to_csv(os.path.join(OUTPUTPATH, 'Sphere_rules.csv'), index=False)
rule_sphere

,rule,type,coef,importance
5596,core_mw_total> 22332.4004 & corona_mw_total<= 12760.5298 & temp> 47.0 & mon_core_mw<= 156.595 & core_mw_total> 15534.3501 & conc> 16.1 & mon_corona_apol<= 42.472 & ratio_mass<= 0.2866 & clogp_core<= 1.37,rule,-3.888100,1.332257
11391,mon_core_apol<= 22.8716 & corona_mv_total<= 92.6556 & ph> 2.625 & corona_mw_total<= 15664.1104 & ratio_mass<= 0.325 & conc> 6.25 & core_mw_total> 27375.2002 & charged<= 0.5 & clogp_core<= 1.37,rule,-1.955195,0.621522
8878,core_mv_total> 135.5229 & core_mv_total<= 338.511 & dp_core> 105.0 & salt<= 0.05 & mon_core_psa> 37.18 & ratio_vol<= 0.3104 & clogp_core<= 1.1175 & dp_corona> 29.5 & corona_mw_total<= 10396.3247 & ratio_mass> 0.2342 & corona_mw_total<= 11925.2148,rule,-1.861998,0.555861
9017,dp_core> 96.5 & core_mv_total<= 510.1229 & mon_corona_apol<= 23.6109 & core_mw_total> 15703.46 & corona_mv_total<= 67.7817 & mon_corona_mv<= 1.344 & dp_corona<= 63.5 & clogp_core<= 1.37 & corona_mw_total<= 11925.2148,rule,-1.606697,0.508979
9594,ratio_vol<= 0.2233 & dp_core> 132.5 & mon_core_mv> 1.0413 & core_mw_total> 21611.625 & dp_corona<= 64.5 & mon_corona_apol<= 27.5791 & clogp_core<= 1.37 & mon_core_mv<= 1.1928,rule,-1.526991,0.493943
10892,core_mw_total> 30761.0801 & mon_corona_mv<= 2.3358 & conc> 15.5 & mon_corona_psa<= 215.065 & ratio_vol<= 0.2996 & mon_corona_mw> 163.9357 & clogp_core> 0.895 & clogp_corona<= -2.73 & conc> 16.1 & core_mw_total> 26510.7197,rule,-2.111469,0.446109
4231,corona_mv_total<= 79.0114 & mon_core_mv<= 1.3954 & corona_mw_total<= 12760.5298 & ratio_mass> 0.1338 & mon_corona_mw> 71.55 & temp> 47.0 & core_mw_total<= 43296.0391 & clogp_corona> -5.23 & ratio_vol<= 0.3115 & core_mv_total> 153.082 & conc> 14.5,rule,-1.413864,0.419645
14889,mon_core_mw<= 145.3096 & ratio_mass<= 0.4342 & core_mv_total<= 124.425 & mon_corona_mv<= 1.2652 & dp_corona<= 63.5 & corona_mw_total<= 6562.8699 & clogp_core<= 1.37,rule,-1.897467,0.392255
12630,dp_core> 95.0 & corona_mv_total<= 98.8696 & ratio_vol<= 0.2225 & ratio_mass<= 0.3023 & dp_corona<= 66.0 & clogp_core<= 1.1022 & dp_core> 87.0 & mon_corona_apol<= 27.5791 & clogp_core<= 1.37,rule,-0.962212,0.326188
3774,dp_corona<= 62.5 & clogp_core<= 1.18 & mon_core_psa<= 47.1448 & ratio_mass<= 0.3959 & core_mw_total> 16403.6704 & ratio_mass<= 0.2115 & mon_corona_apol<= 27.5791 & mon_core_mv> 1.186,rule,-1.225426,0.303337


In [7]:
rule_worm = rule_fit_pcc.fitted_[1].get_rules(all_columns)[['rule','type','coef','importance']]
rule_worm.to_csv(os.path.join(OUTPUTPATH, 'Worm_rules.csv'), index=False)
rule_worm

,rule,type,coef,importance
1593,dp_core> 157.5 & corona_mv_total> 79.0114 & dp_corona> 27.0 & temp> 47.0 & ph> 5.5 & conc> 11.5 & ratio_vol> 0.2135 & dp_corona<= 88.0 & mon_core_mw<= 156.06 & ratio_mass<= 0.4183 & ratio_vol> 0.2022,rule,2.895970,0.829544
5871,corona_mw_total<= 7143.2949 & corona_mv_total<= 69.0254 & corona_mv_total<= 52.4219 & ratio_mass> 0.1534 & ratio_mass<= 0.4161 & dp_core<= 146.5 & core_mw_total<= 43151.2109 & mon_core_psa> 37.18 & clogp_core<= 1.4 & ratio_mass> 0.1356 & mon_corona_apol> 11.8462 & clogp_core<= 1.15,rule,3.558758,0.785465
4328,dp_core> 167.5 & mon_core_psa<= 46.7805 & conc<= 11.25 & clogp_core<= 1.18 & dp_core> 200.5,rule,-1.716966,0.593518
183,dp_core> 146.0 & clogp_corona<= 0.25 & ratio_mass<= 0.227 & dp_corona<= 70.5 & sphere<= 0.5 & mon_corona_apol<= 42.472 & mon_core_psa<= 46.5679,rule,-1.495695,0.509319
6250,ratio_vol> 0.2088 & core_mv_total> 133.3125 & corona_mv_total> 66.2538 & ratio_mass> 0.2346 & sphere<= 0.5 & corona_mw_total<= 16666.9927 & mon_core_apol> 21.0474 & clogp_core<= 1.37,rule,1.467778,0.475332
7647,ratio_mass<= 0.2363 & dp_core> 299.5 & dp_corona> 25.5 & ratio_vol<= 0.19,rule,-0.964274,0.417543
1708,mon_core_apol<= 22.8716 & mon_core_apol> 21.9274 & corona_mv_total> 57.0493 & corona_mv_total<= 78.9749 & temp> 47.0 & mon_corona_mv> 0.613 & ratio_vol<= 0.3409 & core_mv_total<= 330.0225 & ratio_mass> 0.237 & dp_corona> 30.0 & clogp_core<= 1.36 & core_mw_total<= 25933.9502 & ratio_vol> 0.2376,rule,1.400675,0.396327
11624,ratio_vol> 0.1934 & corona_mw_total> 12760.5298 & mon_core_apol> 20.5139,rule,-1.423974,0.344271
315,dp_core> 139.0 & mon_core_mw> 146.8847,rule,-0.930977,0.342485
13528,core_mw_total> 32057.7998 & clogp_corona> -5.23 & core_mv_total> 329.557 & ratio_vol<= 0.2114 & ratio_mass<= 0.1989,rule,-0.769359,0.333142


In [8]:
rule_vesicle = rule_fit_pcc.fitted_[2].get_rules(all_columns)[['rule','type','coef','importance']]
rule_vesicle.to_csv(os.path.join(OUTPUTPATH, 'Vesicle_rules.csv'), index=False)
rule_vesicle

,rule,type,coef,importance
9666,mon_corona_psa> 16.42 & corona_mw_total<= 10526.5049 & mon_corona_mw<= 315.11 & mon_core_psa<= 47.1448 & sphere> 0.5 & core_mw_total> 12447.0,rule,-1.398877,0.625738
3315,sphere<= 0.5 & ph<= 3.85 & corona_mw_total<= 7923.4648 & dp_core> 54.0,rule,1.808579,0.488115
1980,temp> 65.0 & ratio_mass<= 0.2535 & clogp_core<= 1.35 & mon_corona_psa<= 79.32 & core_mw_total> 17671.9951 & conc> 13.75 & sphere<= 0.5,rule,1.697385,0.465618
8285,conc> 7.5 & conc<= 27.5 & core_mw_total<= 35618.0 & sphere> 0.5,rule,-0.855348,0.392370
6861,mon_core_psa> 30.915 & core_mw_total<= 57932.1699 & corona_mw_total> 8503.8901 & corona_mv_total<= 85.0091 & core_mv_total> 241.1475 & mon_core_mv> 1.1171 & ratio_vol<= 0.2301 & mon_core_psa<= 47.1448 & dp_corona<= 63.5,rule,1.275340,0.376912
7158,dp_core<= 167.5 & ph> 4.0,rule,-0.744599,0.331163
6049,conc<= 12.75 & mon_corona_psa<= 130.3 & ratio_vol> 0.149 & charged> 0.5,rule,-1.025769,0.320518
9015,mon_core_mw> 142.8175 & corona_mw_total<= 14063.4102 & mon_corona_psa> 47.97 & ratio_mass> 0.0551 & sphere<= 0.5 & clogp_core<= 1.37 & worm<= 0.5,rule,0.944727,0.308509
613,temp> 53.0 & core_mv_total> 191.7303 & core_mw_total<= 43889.8555 & core_mw_total> 31480.9297 & corona_mv_total> 49.6898 & dp_corona> 29.5 & mon_core_mv> 1.1777 & dp_corona<= 64.5 & ratio_mass> 0.1339,rule,1.028269,0.296998
9887,dp_core<= 401.5 & core_mv_total> 245.979 & dp_core> 255.5 & conc<= 17.25 & dp_corona> 50.0 & charged<= 0.5 & clogp_core<= 1.1625 & ratio_vol<= 0.2219,rule,1.424321,0.291220


In [9]:
rule_other = rule_fit_pcc.fitted_[3].get_rules(all_columns)[['rule','type','coef','importance']]
rule_other.to_csv(os.path.join(OUTPUTPATH, 'Other_rules.csv'), index=False)
rule_other

,rule,type,coef,importance
3117,dp_core> 61.5 & salt<= 0.2 & clogp_corona<= 0.73 & corona_mw_total<= 14338.48 & mon_corona_mv> 0.7838 & conc> 7.5,rule,-2.253141,0.985906
2071,dp_core> 127.0 & corona_mw_total> 5957.905 & core_mw_total<= 30217.0 & mon_corona_psa<= 65.0 & sphere<= 0.5,rule,4.660336,0.583991
114,sphere> 0.5,rule,-1.097951,0.547035
1689,mon_corona_mv<= 0.7838 & clogp_corona> -5.23 & conc> 18.75 & sphere<= 0.5 & core_mv_total<= 284.4 & salt<= 0.2,rule,4.733872,0.421713
230,mon_core_psa<= 46.5679 & core_mv_total> 261.2925,rule,-0.718434,0.357242
2430,corona_mw_total<= 5113.05 & mon_core_mw<= 151.085 & ratio_mass<= 0.3479 & conc<= 18.75 & corona_mw_total<= 14338.48,rule,-1.270576,0.345718
1495,ratio_vol> 0.1885 & mon_corona_psa<= 79.32 & ratio_vol<= 0.2363 & mon_corona_apol> 23.4875 & corona_mv_total> 39.607 & mon_corona_mw> 162.1158 & sphere<= 0.5,rule,3.879928,0.341598
2366,conc> 17.5 & clogp_corona> 0.125 & core_mw_total> 18010.0 & corona_mv_total<= 107.5755 & mon_core_apol<= 24.2743 & core_mw_total<= 33858.7998,rule,3.034554,0.311321
2914,conc> 11.25 & dp_corona<= 120.0 & clogp_corona<= 0.73 & mon_core_psa<= 46.5679 & mon_corona_mv> 0.7838,rule,-0.616037,0.307249
2315,vesicle<= 0.5 & corona_mw_total<= 6338.1602 & core_mw_total<= 23694.4502,rule,-0.939464,0.285106


__Get all morphology rules__


In [10]:
# get all rules:
sphere_top5_rule = rule_sphere
worm_top5_rule = rule_worm
vesicle_top5_rule = rule_vesicle

sphere_top5_rule.reset_index(drop=True, inplace=True)
worm_top5_rule.reset_index(drop=True, inplace=True)
vesicle_top5_rule.reset_index(drop=True, inplace=True)

top5_rule = pd.concat([sphere_top5_rule, worm_top5_rule, vesicle_top5_rule], axis=1)
upper_columns = ['Sphere', 'Worm', 'Vesicle']
lower_columns = ['rule', 'type', 'coef', 'importance']

top5_rule.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
top5_rule.to_csv(os.path.join(OUTPUTPATH, 'Combine_All_Rules.csv'), index=False)
top5_rule

Phase                                                                                                                                                                                                                                                                                                                                       Sphere  \
Property                                                                                                                                                                                                                                                                                                                                      rule   
0         core_mw_total> 22332.4004 & corona_mw_total<= 12760.5298 & temp> 47.0 & mon_core_mw<= 156.595 & core_mw_total> 15534.3501 & conc> 16.1 & mon_corona_apol<= 42.472 & ratio_mass<= 0.2866 & clogp_core<= 1.37                                                                                                                                
1         mon_core_apol<= 22.8716 & corona_mv_total<= 92.6556 & ph> 2.625 & corona_mw_total<= 15664.1104 & ratio_mass<= 0.325 & conc> 6.25 & core_mw_total> 27375.2002 & charged<= 0.5 & clogp_core<= 1.37                                                                                                                                           
2         core_mv_total> 135.5229 & core_mv_total<= 338.511 & dp_core> 105.0 & salt<= 0.05 & mon_core_psa> 37.18 & ratio_vol<= 0.3104 & clogp_core<= 1.1175 & dp_corona> 29.5 & corona_mw_total<= 10396.3247 & ratio_mass> 0.2342 & corona_mw_total<= 11925.2148                                                                                     
3         dp_core> 96.5 & core_mv_total<= 510.1229 & mon_corona_apol<= 23.6109 & core_mw_total> 15703.46 & corona_mv_total<= 67.7817 & mon_corona_mv<= 1.344 & dp_corona<= 63.5 & clogp_core<= 1.37 & corona_mw_total<= 11925.2148                                                                                                                   
4         ratio_vol<= 0.2233 & dp_core> 132.5 & mon_core_mv> 1.0413 & core_mw_total> 21611.625 & dp_corona<= 64.5 & mon_corona_apol<= 27.5791 & clogp_core<= 1.37 & mon_core_mv<= 1.1928                                                                                                                                                             
5         core_mw_total> 30761.0801 & mon_corona_mv<= 2.3358 & conc> 15.5 & mon_corona_psa<= 215.065 & ratio_vol<= 0.2996 & mon_corona_mw> 163.9357 & clogp_core> 0.895 & clogp_corona<= -2.73 & conc> 16.1 & core_mw_total> 26510.7197                                                                                                              
6         corona_mv_total<= 79.0114 & mon_core_mv<= 1.3954 & corona_mw_total<= 12760.5298 & ratio_mass> 0.1338 & mon_corona_mw> 71.55 & temp> 47.0 & core_mw_total<= 43296.0391 & clogp_corona> -5.23 & ratio_vol<= 0.3115 & core_mv_total> 153.082 & conc> 14.5                                                                                     
7         mon_core_mw<= 145.3096 & ratio_mass<= 0.4342 & core_mv_total<= 124.425 & mon_corona_mv<= 1.2652 & dp_corona<= 63.5 & corona_mw_total<= 6562.8699 & clogp_core<= 1.37                                                                                                                                                                       
8         dp_core> 95.0 & corona_mv_total<= 98.8696 & ratio_vol<= 0.2225 & ratio_mass<= 0.3023 & dp_corona<= 66.0 & clogp_core<= 1.1022 & dp_core> 87.0 & mon_corona_apol<= 27.5791 & clogp_core<= 1.37                                                                                                                                              
9         dp_corona<= 62.5 & clogp_core<= 1.18 & mon_core_psa<= 47.1448 & ratio_mass<= 0.3959 & core_mw_total> 16403.6704 & ratio_mass<= 0.2115 & mon_corona_apol<= 27.5791 & mon_core_mv> 1.186                                              

__Get the Top5 morphology rules__


In [11]:
# get top 5:
sphere_top5_rule = rule_sphere.iloc[:5, ]
worm_top5_rule = rule_worm.iloc[:5, ]
vesicle_top5_rule = rule_vesicle.iloc[:5, ]

sphere_top5_rule.reset_index(drop=True, inplace=True)
worm_top5_rule.reset_index(drop=True, inplace=True)
vesicle_top5_rule.reset_index(drop=True, inplace=True)

top5_rule = pd.concat([sphere_top5_rule, worm_top5_rule, vesicle_top5_rule], axis=1)
upper_columns = ['Sphere', 'Worm', 'Vesicle']
lower_columns = ['rule', 'type', 'coef', 'importance']

top5_rule.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
top5_rule.to_csv(os.path.join(OUTPUTPATH, 'Combine_Top5_Rules.csv'), index=False)
top5_rule

Phase                                                                                                                                                                                                                                                     Sphere  \
Property                                                                                                                                                                                                                                                    rule   
0         core_mw_total> 22332.4004 & corona_mw_total<= 12760.5298 & temp> 47.0 & mon_core_mw<= 156.595 & core_mw_total> 15534.3501 & conc> 16.1 & mon_corona_apol<= 42.472 & ratio_mass<= 0.2866 & clogp_core<= 1.37                                              
1         mon_core_apol<= 22.8716 & corona_mv_total<= 92.6556 & ph> 2.625 & corona_mw_total<= 15664.1104 & ratio_mass<= 0.325 & conc> 6.25 & core_mw_total> 27375.2002 & charged<= 0.5 & clogp_core<= 1.37                                                         
2         core_mv_total> 135.5229 & core_mv_total<= 338.511 & dp_core> 105.0 & salt<= 0.05 & mon_core_psa> 37.18 & ratio_vol<= 0.3104 & clogp_core<= 1.1175 & dp_corona> 29.5 & corona_mw_total<= 10396.3247 & ratio_mass> 0.2342 & corona_mw_total<= 11925.2148   
3         dp_core> 96.5 & core_mv_total<= 510.1229 & mon_corona_apol<= 23.6109 & core_mw_total> 15703.46 & corona_mv_total<= 67.7817 & mon_corona_mv<= 1.344 & dp_corona<= 63.5 & clogp_core<= 1.37 & corona_mw_total<= 11925.2148                                 
4         ratio_vol<= 0.2233 & dp_core> 132.5 & mon_core_mv> 1.0413 & core_mw_total> 21611.625 & dp_corona<= 64.5 & mon_corona_apol<= 27.5791 & clogp_core<= 1.37 & mon_core_mv<= 1.1928                                                                           

Phase                                \
Property  type      coef importance   
0         rule -3.888100  1.332257    
1         rule -1.955195  0.621522    
2         rule -1.861998  0.555861    
3         rule -1.606697  0.508979    
4         rule -1.526991  0.493943    

Phase                                                                                                                                                                                                                                                                                           Worm  \
Property                                                                                                                                                                                                                                                                                        rule   
0         dp_core> 157.5 & corona_mv_total> 79.0114 & dp_corona> 27.0 & temp> 47.0 & ph> 5.5 & conc> 11.5 & ratio_vol> 0.2135 & dp_corona<= 88.0 & mon_core_mw<= 156.06 & ratio_mass<= 0.4183 & ratio_vol> 0.2022                                                                                      
1         corona_mw_total<= 7143.2949 & corona_mv_total<= 69.0254 & corona_mv_total<= 52.4219 & ratio_mass> 0.1534 & ratio_mass<= 0.4161 & dp_core<= 146.5 & core_mw_total<= 43151.2109 & mon_core_psa> 37.18 & clogp_core<= 1.4 & ratio_mass> 0.1356 & mon_corona_apol> 11.8462 & clogp_core<= 1.15   
2         dp_core> 167.5 & mon_core_psa<= 46.7805 & conc<= 11.25 & clogp_core<= 1.18 & dp_core> 200.5                                                                                                                                                                                                  
3         dp_core> 146.0 & clogp_corona<= 0.25 & ratio_mass<= 0.227 & dp_corona<= 70.5 & sphere<= 0.5 & mon_corona_apol<= 42.472 & mon_core_psa<= 46.5679                                                                                                                                              
4         ratio_vol> 0.2088 & core_mv_total> 133.3125 & corona_mv_total> 66.2538 & ratio_mass> 0.2346 & sphere<= 0.5 & corona_mw_

In [29]:
from copy import deepcopy
import re

def countNumber(data, rules):
    rules = [each.strip() for each in rules.split('&')]
    copy_data = deepcopy(data)
    for each in rules:
        subrules = re.split("(>=|<=|<|=|>)", each)
        subrules = [each.strip() for each in subrules]
        rv, symb, val = subrules
        if ">=" in each:
            copy_data = copy_data[copy_data[rv] >= float(val)]
        elif "<=" in each:
            copy_data = copy_data[copy_data[rv] <= float(val)]
        elif ">" in each:
            copy_data = copy_data[copy_data[rv] > float(val)]
        elif "<" in each:
            copy_data = copy_data[copy_data[rv] < float(val)]
        else:
            copy_data = copy_data[copy_data[rv] == float(val)]
    return copy_data.shape[0]

